## 🐶End-to-end Multi-class Dog Breed Classification using colab

This notebook builds an end-to-end multi-class image classifier using TensorFlow and TensorFlow Hub.

## 1.Problem

Identifying the breed of a dog given an image of a dog.

When I'm sitting at the cafe and I take a photo of a dog, want to know what bredd of dof it is.

## 2.Data

The data we're using is from Kaggle's dog breed identication competition.
https://www.Kaggle.com/c/dog-breed-identification/data

## 3.Evaluation

The evalutation is a file with prediction probabilites for each dog breed of each test image.
https://www.kaggle.com/c/dog-breed-identification/overview/evaluation

## 4.Features

Some information about the data:
* we're deakubg with `image`(unstructured data) so it's probably best we use deep learning/transfer learning.
* There are 120 breeds of dogs(this means there are 120 different classes).
* There are around 10,000+ images in the training set(these images have labels).
* There are around 10,000+ images in the test set(these have no labels,because we'll want to predict them).


## Get our workspace ready

In [ ]:
# Import TensorFlow into Colab
import tensorflow as tf
print("TF version", tf.__version__)
import tensorflow_hub as hub
print("TF Hub version", hub.__version__)

# Check for GPU availability
print("GPU", "available (YESSSSSSSSSSS!!!!)" if tf.config.list_physical_devices("GPU") else "not available")

 ## Getting o ur data ready (turning into Tensors)
 with all machine learning models, our data has to be in numerical format. So that's what i will be doing first. Truning our images into TEnsors(numerical representation).
 Let's start by accessing our data and checking out the labels.

In [ ]:
# checkout the labels of our data
import pandas as pd
labels_csv = pd.read_csv("drive/MyDrive/dog-vision/dog-breed-identification.zip (Unzipped Files)/labels.csv")
print(labels_csv.describe())
print(labels_csv.head())

In [ ]:
labels_csv.head()

In [ ]:
# how many images are there of each breed?
labels_csv["breed"].value_counts().plot.bar(figsize=(20,10))

In [ ]:
labels_csv["breed"].value_counts().median()

In [ ]:
from IPython.display import Image
Image("drive/MyDrive/dog-vision/dog-breed-identification/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg")

### Getting images and their labels
getting a list of all of the images file pathnames.

In [ ]:
# Create pathnames form image ID"s
filename = ["drive/MyDrive/dog-vision/dog-breed-identification/train/" + fname + ".jpg" for fname in labels_csv["id"]]
filename[:10]

In [ ]:
# Check whether number of filenames matches number of actual images file

import os
if len(os.listdir("drive/MyDrive/dog-vision/dog-breed-identification/train/")) == len(filename):
  print("Filename match actual amout of files!!! proceed")
else:
  print("Filename do not mach with actual amount of files")

In [ ]:
#one more check
Image(filename[9000])

In [ ]:
labels_csv['breed'][9000]

Since i now got my training images filepath in a list, prepare my labels.

In [ ]:
import numpy as np
labels = labels_csv["breed"].to_numpy()
labels

In [ ]:
# see if number of lables matches the number of filenames
if len(labels) == len(filename):
  print("Number matches")
else:
  print("Number don't matches")

In [ ]:
# find the unique label values
unique_breeds= np.unique(labels)
len(unique_breeds)

In [ ]:
# Trun a single labels into an array of  booleans
print(labels[0])
labels[0] == unique_breeds

In [ ]:
# Turn every labels into a booleans array
boolean_labels = [label == unique_breeds for label in labels]
boolean_labels[:2]

In [ ]:
# Example: turning boolean array into integers
print(labels[0])
print(np.where(unique_breeds == labels[0]))# index where label occurs
print(boolean_labels[0].argmax())# index where labels occures in boolean array
print(boolean_labels[0].astype(int))# there will be a 1 where the sample label occurs

### Creating our own validation set
since the dataset from kaggle doesn't come with a validation set, i am going to create my own.

In [ ]:
# Setup x and y variable
x = filename
y = boolean_labels

I am going to start off experimenting with ~1000 images and increase as needed.

In [ ]:
# set number of images to use for experimenting
NUM_IMAGES = 1000 #@param {type:"slider", min:1000, max:10000, step:1000}

In [ ]:
# spliting data into train and validation set
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x[:NUM_IMAGES],
                                                  y[:NUM_IMAGES],
                                                  test_size=0.2,
                                                  random_state=42)
len(x_train), len(y_train), len(x_val), len(y_val)

In [ ]:
# Lets have a geez the training data
x_train[:5], y_train[:2]

## Preprocessing images(tuning images into Tensors)
To preprocess our images into Tensors we're going to write a function which does a few things:
1. Take an image filepath as input
2. Use TensorFlow to read the file and save it to a variable, `image`
3. Turn our `image` (a jpg) into Tensors
4. Normalize our image(convert color channel values form 0-225 to 0-1).
5. Resize the `image` to be a shape of (224,224)
6. Return the modified `image`

In [ ]:
# convert images to numpy array
from matplotlib.pyplot import imread
image = imread(filename[42])
image.shape

In [ ]:
tf.constant(image[:2])

In [ ]:
# Define image size
IMG_SIZE = 224

#Create a function for preprocessing images
def process_image(image_path, img_size= IMG_SIZE):
  """
  Takes an image file path and turns hte image into a Tensor.
  """
  #Read in an image file
  image = tf.io.read_file(image_path)
  #Turn the jpeh image into numerical Tensor with 3 colour channels (Red, Green,Blue)
  image = tf.image.decode_jpeg(image, channels=3)
  # conver the colour channel values to from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  #Resize the image to desired value (224, 224)
  image = tf.image.resize(image, size=[IMG_SIZE,IMG_SIZE])

  return image


## Turning our data into batches
Why turn our data into batches?

Let's say you're trying to process 10,000+ images in one go... they all might not fit into memory.

So that's why we do about 32 (this is the batch size) images at a time (you can manually adjust the batch size if need be).

In order to use TensorFlow effectively, we need our data in the form of Tensor tuples which look like this:
`(image, label)`.


In [ ]:
# Create a simple funtion to return a tuple (image, label)

def get_image_label(image_path, label):
  """
  Takes an image file path name and the assosciated label,
  processes the image and return a tuple of (image, label).
  """
  image = process_image(image_path)
  return image, label

In [ ]:
(process_image(x[42], tf.constant(y[42])))

Now we've got a way to turn our data into tuples of Tensors in the form:`(image, label)` let's make a function to turn all of our data (`x`& `y`) into batches!

In [ ]:
# Define the batch size, 32 is a good start
BATCH_SIZE = 32
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
    creates batches of data out of image (x) and label (y) pairs.
    shuffles the data if it's training data but doesn't shuffle if it's validation data.
    also accepts test data as input(no labels).
  """
  #if the data is a test dataset, we probably don't have have labels
  if test_data:
    print("Creating test data batches....")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) #only filepath (No labels)
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  # if the data is a valid dataset, dwe don't need to shuffle it
  elif valid_data:
    print("Creating validation data batches....")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x),# filepaths
                                               tf.constant(y)))#labels
    data_batch = data.map(get_image_label).batch(BATCH_SIZE) # Use get_image_label to get both image and label
    return data_batch

  else:
    print("Creating training data batches....")
    #Trun filepaths and labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x),
                                               tf.constant(y)))
    # shuffling pathnames and labels before mapping image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(x))
    # create (image, label) tuples (this also turns the image path into a preprocessed image)
    data = data.map(get_image_label)
    #turn the training data into baches
    data_batch = data.batch(BATCH_SIZE)
  return data_batch

In [ ]:
# Create training and validation data batches
train_data = create_data_batches(x_train, y_train)
val_data = create_data_batches(x_val, y_val, valid_data=True)

In [ ]:
# Check out the differenct attribute of our data batches
train_data.element_spec, val_data.element_spec

## Visualizing data batches

our data is now in batches however these can be a little hard ot understand/comperhand, let's zisualize them!

In [ ]:
import matplotlib.pyplot as plt

# create a fuction for viewing images in a data batch
def show_25_images(image, labels):
  """
  Displays 25 images from a data batch.
  """
  plt.figure(figsize=(10,10))
  for i in range(25):
    # create subplots (5 rows, 5 columns)
    ax = plt.subplot(5,5, i+1)
    plt.imshow(image[i])
    plt.title(unique_breeds[labels[i].argmax()])
    #turn the grid line off
    plt.axis("off")


In [ ]:
# Now let's visualize the data in a training batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

## building a model

Before we build a model, there are a few things we need to define:
* The  input shape (our images shape, in the form of Tensore) to our model.
* The output shape (images labels, in the form of Tensors) of our model.
* The URL of the model we want to use from TensorFlow Hub-https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4.

NOw we've got our inputs, outputs and model ready to go.
Let's put htem together into a keras deep learning model!

Knowing this , let's create a function which:
* Takes  the input shape, output shape and the modle we've chosen as parameters.
* Define the layers in  keras model  in seuential fashion(do this first, then this, then that)
* Compiles the model(says it should be evaluated and improved)
* Builds the model (tells the model the input shape it'll be getting)
* Returns the model.

In [ ]:
#build a function to train and return a trained model
def train_model():
    """
    Train a given model and retrun the trained version.
    """
    # Create a model
    model = create_model()

    #create new TesnorBoard session everytime we train a model
    tensorboard = create_tensorboard_callback()
    # Fit the model to the data passing it the call backs we created
    model.fit(x=train_data,
              epochs=NUM_EPOCHS,
              validation_data=val_data,
              validation_freq=1,
              callbacks=[tensorboard, early_stopping])
    # return the fitted model
    return model


## Createing callbacks

callbacks are helper function a modle can use during training to do such things as save its progress,check its progress or stop training early if a model stop improving.

we'll create two callbacks, one for TensorBoard which helps track our models progress and another for early stopping which prevents out model from training for too long.
## TensorBoard Callback

To setup a TesnorBoard callback, we need to do 3 things:
1. Load the TensorBoard notebook extension.
2. create a TensorBoard callback which is able to save logs to a directory and pass it to our model's `fit()` function.
3. Visualize our models training logs with the `%tensorboard` magic function(we'll do this after model training).

In [ ]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import datetime
# crate a function to build  a Tenosrboard callback
def create_tensorboard_callback():
  #create a log directory to track logs when ever it runs
  logdir = os.path.join("drive/MyDrive/dog-vision/dog-breed-identification/logs",
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

## Early Stopping callback
Early stopping helps stop our model from overfitting by stopping training if a certain evaluration metric stops improving.

In [ ]:
# create early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3)

## Training a model (on subset of data)

Our fist model is only going to train on 1000 images, to make sure everything is working.

In [ ]:
NUM_EPOCHS = 100 #@param {type:"slider", min:10, max:100, step:10}

In [ ]:
# Check to make sure we're still running on a GPU
print("GPU", "available (YESSSSSSSSSSS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

In [ ]:
INPUT_SHAPE = (224, 224, 3)
OUTPUT_SHAPE = len(unique_breeds)

def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE):
    print("Building model with tf.keras.applications.EfficientNetB0")

    input_layer = tf.keras.layers.Input(shape=input_shape)

    # Use built-in EfficientNetB0 (pre-trained on ImageNet, feature extractor mode)
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False,  # Remove the top classification layers
        weights='imagenet',
        input_shape=input_shape,
        pooling='avg'  # Global average pooling to get a feature vector
    )(input_layer)
    base_model.trainable = False  # Freeze the base

    # Add output layer
    output_layer = tf.keras.layers.Dense(units=output_shape, activation="softmax")(base_model)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"]
    )

    return model

model = create_model()
model.summary()


 the model is overfitting which is good thing to start with

In [ ]:
model = train_model()

### Checking the TesnorBoard logs

The TesnorBoard magic function(%tensorboard) will access he logs directory we created earlier and visualize its contents.

In [ ]:
%tensorboard --logdir drive/MyDrive/dog-vision/dog-breed-identification/logs

## Making and evaluating prediction using a trained model

In [ ]:
# Make prediction on the validation data (not used to train on)
predictions =  model.predict(val_data, verbose=1)
predictions

In [ ]:
predictions.shape

In [ ]:
predictions[81]

In [ ]:
unique_breeds[np.argmax(predictions[20])]

In [ ]:
predictions[0]

In [ ]:
np.sum(predictions[0])

In [ ]:
# First prediction
index = 97
print(predictions[index])
print(f"Max value (probability of predictioni): {np.max(predictions[index])}")
print(f"Sum: {np.sum(predictions[index])}")
print(f"Max index: {np.argmax(predictions[index])}")
print(f"predicted label: {unique_breeds[np.argmax(predictions[index])]}")

In [ ]:
unique_breeds[63]

Having the above funtionality is greate but we want to be able to do it at scale.

And it would be even better if we could see the image the prediction is being made on!

**Note**: prediction probabilities are also known as confidence levels.

In [ ]:
#Trun prediction probablity into their respective label (easier to understand)
def get_pred_label(predictions_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return unique_breeds[np.argmax(predictions_probabilities)]
# Get a predicted label based on an array of predictions probabilities
pred_label = get_pred_label(predictions[90])
pred_label

In [ ]:
unique_breeds[8]

In [ ]:
val_data


Now since our validation data is still in a batch dataset,
we'll have to unbatchify it to make prediction on the validation images and then compaer those prediction to the validation labels(true labels).

In [ ]:
# create a function to unbatch a batch dataset
def unbatchify(data):
  """
  Takes a batched dataset of (images, label) Tensor and retrun separate arrays of image and labels.
  """
  images= []
  labels= []
  # loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique_breeds[np.argmax(label)])
  return images, labels

# Unbatchify the validation data
val_images, val_labels = unbatchify(val_data)
val_images[0], val_labels[0]

In [ ]:
get_pred_label(val_images[0])

Now we've got ways to get:
  * prediction labels
  * validation labels(train_data)
  * validation images

Let's make some functions to make these all a bit more visualize.

we'll create a function which:
* Take an array of predictions probabilities, an array of truth labels and un array of images and integers.
* convert the prediction probabilities to predicted label.
* Plot the predicted lable,it's predicted probability, the truth label and the target image on s single plot.

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=1):
  """
  View the prediction, ground truth and image for sample n
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n],images[n]

  #Get the pred label
  pred_label = get_pred_label(pred_prob)

  # Plot image and remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  #change plot title to predicted
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  # change plot titile to be predicted, probabiltiy of prediction and truth label
  plt.title("{} {:2.0f}% {}".format(pred_label,
                                    np.max(pred_prob)*100,
                                    true_label),
                                    color = color
            )


In [ ]:
plot_pred(prediction_probabilities=predictions,
          labels = val_labels,
          images = val_images,
          n=77)

Now we've got one function to visualize our modle top predictions, let's make another to view our model top 10 predictions.

This function will:
* Take an input of prediction probabilites array and an integer
* Find the prediction using get_pred_label()
* find the top 10

In [ ]:
def plot_pred_conf(prediction_probabilities,labels, n=1):
  """
  Plot the top 10 highest prediction confidences along with the truth label for sample n.
  """
  pred_prob, true_label = prediction_probabilities[n], val_labels[n]

  # Get the predicted label
  pred_label = get_pred_label(pred_prob)

  # Find the top 10 prediction
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]
  # find the top 10 prediction confidence values
  top_10_pred_values = pred_prob[top_10_pred_indexes]
  # Find the top 10 prediction labels
  top_10_pred_labels = unique_breeds[top_10_pred_indexes]

  # Setup plot
  top_plot = plt.bar(np.arange(len(top_10_pred_labels)),
                     top_10_pred_values,
                     color = "grey")
  plt.xticks(np.arange(len(top_10_pred_labels)),
             labels=top_10_pred_labels,
             rotation="vertical")

  # Change color of true label
  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass


In [ ]:
plot_pred_conf(prediction_probabilities=predictions,
               labels=val_labels,
               n= 9)

Now we've got some function to help us visualize our predictions and evaluate our model, let's check out..

In [ ]:
#lets check out a few predictions and their different values
i_multiplier = 0
num_rows = 3
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(10*num_cols, 5*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_pred(prediction_probabilities=predictions,
            labels=val_labels,
            images=val_images,
            n=i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(prediction_probabilities=predictions,
                 labels=val_labels,
                 n=i+i_multiplier)
plt.tight_layout(h_pad=1.0)
plt.show()

##Saving and reloading a trained model

In [ ]:
# Create a function to save a model
def save_model(model, suffix=None):
  """
  saves a given model in a models directory and appends a suffix (string).
  """

  #Create a model directory pathname with current time
  modeldir = os.path.join("drive/MyDrive/dog-vision/dog-breed-identification/models",
                       datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  model_path = modeldir + "-" + suffix + ".h5"
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

In [ ]:
# create a fuction to load aa trained model
def load_model(model_path):
  """
  Load a saved model from a specified path.
  """

  print(f"Loading saved model from: {model_path}")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={"kerasLayer":hub.kerasLayer})
  return model

Now we've got function to save and load a trined model, let's make sure they work!


In [ ]:
# save our model trained on 1000 images
save_model(model, suffix="1000-images-mobilenetv2-adam")